In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision
from PIL import Image
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split

In [ ]:
def load_image_paths(root_dir):
    original_dir = os.path.join(root_dir, "original")
    degraded_dir = os.path.join(root_dir, "degraded")
    
    original_images = [os.path.join(original_dir, img) for img in os.listdir(original_dir)]
    degraded_images = [os.path.join(degraded_dir, img) for img in os.listdir(degraded_dir)]
    print(degraded_images[:5])
    # Mapping original to degraded
    mapping = {}
    for orig_path in original_images:
        base_name = os.path.basename(orig_path)
        # print(base_name)
        idx, barcode, _ = base_name.split('_')
        # print([img for img in degraded_images])
        degraded_versions = [img for img in degraded_images if img.startswith(f"./data/tmp/degraded/{idx}_{barcode}")]
        mapping[orig_path] = degraded_versions
    
    return mapping

# Assuming '/data/tmp' is your root directory
root_dir = './data/tmp'
image_paths = load_image_paths(root_dir)

# Splitting into train and test sets
original_images = list(image_paths.keys())
train_orig, test_orig = train_test_split(original_images, test_size=0.2, random_state=42)

# Creating train and test mappings
train_mapping = {orig: image_paths[orig] for orig in train_orig}
test_mapping = {orig: image_paths[orig] for orig in test_orig}

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, mapping, transform=None):
        self.mapping = mapping
        self.transform = transform
        self.original_images = list(mapping.keys())
    
    def __len__(self):
        return len(self.original_images)
    
    def __getitem__(self, idx):
        orig_path = self.original_images[idx]
        degraded_paths = self.mapping[orig_path]
        
        # Example: loading the first degraded image
        degraded_path = random.choice(degraded_paths) # Randomly select one for variability
        orig_image = Image.open(orig_path)
        degraded_image = Image.open(degraded_path)
        
        if self.transform:
            orig_image = self.transform(orig_image)
            degraded_image = self.transform(degraded_image)
        
        return degraded_image, orig_image

In [ ]:
# Define a function to convert images to binary
def to_binary(tensor, threshold=0.5):
    # Convert to grayscale by averaging the channels
    grayscale = tensor.mean(dim=0, keepdim=True)
    # Apply threshold
    binary = (grayscale > threshold).float()
    return binary

# Create the transform pipeline
transform = transforms.Compose([
    transforms.Resize((256, 512)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: to_binary(x))
])

train_dataset = ImageDataset(train_mapping, transform=transform)
test_dataset = ImageDataset(test_mapping, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to convert a tensor to a numpy array
def imshow(img):
    img = img.numpy() # Convert tensor to numpy array
    plt.imshow(np.transpose(img, (1, 2, 0))) # Convert from CxHxW to HxWxC
    plt.show()

# Get a batch of training images
dataiter = iter(train_loader)
images, labels = next(dataiter) # Use next() to get the next batch

# Show images for the first image in the batch
print(images[0].unsqueeze(0).shape)
imshow(torchvision.utils.make_grid(images[0].unsqueeze(0)))

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
         
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = Autoencoder()

In [ ]:
for data in train_loader:
    print(data[0].shape)
    print(data[0][0].shape)
    break

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
 
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
 
# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0
    num_batches = 0
    
    loop = tqdm(train_loader, leave=True)
    for degraded_images, original_images in loop:
        degraded_images = degraded_images.to(device)
        original_images = original_images.to(device)
        
        reconstructed_images = model(degraded_images)
        loss = criterion(reconstructed_images, original_images)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
    
    average_loss = total_loss / num_batches
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}")
    
    # Evaluate every 10 epochs
    if (epoch + 1) % 10 == 0:
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():  # Inference mode, gradients not calculated
            val_loss = 0
            val_batches = 0
            for degraded_images, original_images in test_loader:
                degraded_images = degraded_images.to(device)
                original_images = original_images.to(device)
                
                reconstructed_images = model(degraded_images)
                loss = criterion(reconstructed_images, original_images)
                
                val_loss += loss.item()
                val_batches += 1
            
            average_val_loss = val_loss / val_batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {average_val_loss:.4f}")

# Save the model
torch.save(model.state_dict(), 'model.pth')